In [1]:
import pandas as pd
import numpy as np
import json
import sys
import random

import ast

import pymysql

from datetime import date

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import matplotlib.patheffects as path_effects
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import to_rgba
from matplotlib import cm
from matplotlib import colorbar
import matplotlib as mpl
import matplotlib.patches as patches
import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.patches import RegularPolygon
from matplotlib.patches import ArrowStyle
from matplotlib.patches import Circle

from matplotlib.colors import Normalize
import matplotlib.patheffects as pe

import plotly.express as px
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

from mplsoccer import VerticalPitch, Pitch, Radar, FontManager, grid, PyPizza

import scipy.stats as stats

from highlight_text import  ax_text, fig_text

from soccerplots.utils import add_image

from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import preprocessing

import abs_import

import statistics
import math
import ipywidgets as widgets

from IPython.display import display, Math, Latex

import warnings

plt.rcParams["figure.dpi"] = 300

from matplotlib import font_manager

from app import *

sys.path.append('Functions/visualization')

from Functions.utils import read_json
from Functions.visualization.passing_network import draw_pitch, draw_pass_map

clubColors = {'Brazil' : ['#fadb04', '#1c3474'],
              'Portugal' : ['#e1231b', '#004595'],
              'Argentina' : ['#52a9dc', '#dbe4ea'],
              'Saudi Arabia' : ['#145735', '#dbe4ea'],
              'Ghana' : ['#145735', '#dbe4ea'],
              'Serbia' : ['#FF0000', '#E8E8E8'],
              'Spain' : ['#FF0000', '#E8E8E8'],
              'Germany' : ['#aa9e56', '#FF0000'],
              'France' : ['#202960', '#d10827'],
              'Poland' : ['#d10827', '#E8E8E8'],
              'Morocco' : ['#db221b', '#044c34'],
              'Croatia' : ['#e71c23', '#3f85c5'],
              'Netherlands' : ['#f46c24', '#dcd9d7'],
              'Senegal' : ['#34964a', '#eedf36'],
              'Denmark' : ['#cb1617', '#E8E8E8'],
              'Iran' : ['#269b44', '#dd1212'],
              'Belgium' : ['#ff0000', '#e30613'],
              'USA' : ['#ff0000', '#202960'],
              'Switzerland' : ['#ff0000', '#e8e8e8'],
              'Australia' : ['#202960', '#e30613'],
              'Wales' : ['#ff0000', '#e8e8e8'],
              'Mexico' : ['#00a94f', '#ff0000'],
              'Uruguay' : ['#52a9dc', '#e8e8e8'],
              'Canada' : ['#ff0000', '#ff0000'],
              'Costa Rica' : ['#ff0000', '#202960']}

df1 = pd.read_csv('homeGameformations.csv')
df1.rename({'playerIds' : 'playerId'}, axis=1, inplace=True)
df2 = pd.read_csv('optaMundial.csv')
df2["matchTimestamp"] = 60 * df2["minute"] + df2["second"]
df2["matchTimestamp"] = pd.to_datetime(df2["matchTimestamp"], unit='s')
df2.drop_duplicates(subset=['name', 'matchTimestamp', 'team', 'typedisplayName', 'x', 'y'], keep='first', inplace=True)
df2.sort_values(by=['Match_ID', 'matchTimestamp'], inplace=True, ascending=[True, True])
df2.reset_index(drop=True, inplace=True)

In [22]:
# CREATE FUNCTION TO SPLIT VALUES FROM A LIST SEPARETED WITH ,
def pad_list(lst, max_len):
    return lst + [lst[-1]] * (max_len - len(lst))

# SELECT THE COLUMNS WE WANT TO TRANSFORM
df3 = df1[['formationId', 'jerseyNumbers', 'formationName', 'playerId', 'formationSlots', 'startMinuteExpanded', 'endMinuteExpanded']]

# CONVERT COLUMNS TO STRING
df3[['startMinuteExpanded', 'endMinuteExpanded', 'formationId']] = df3[['startMinuteExpanded', 'endMinuteExpanded', 'formationId']].astype(str)

# APPLY FUNCTION TO SPLIT VALUES FROM A LIST SEPARETED WITH , AND EXPLODE THEM AND CONVERT TO LIST
new_df = df3[['formationSlots', 'startMinuteExpanded', 'endMinuteExpanded', 'formationId', 'jerseyNumbers', 'playerId']].apply(lambda row: row.apply(lambda item: pad_list(item.split(','), max(row.str.contains(',') + 1))), axis=1).explode(df3[['formationSlots', 'jerseyNumbers', 'playerId']].columns.tolist())

# REMOVE ONE CARATHER FROM INSIDE A COLUMN VALUE
new_df['formationSlots'] = new_df['formationSlots'].str.replace("[[]","")
new_df['formationSlots'] = new_df['formationSlots'].str.replace("[]]","")

new_df['jerseyNumbers'] = new_df['jerseyNumbers'].str.replace("[[]","")
new_df['jerseyNumbers'] = new_df['jerseyNumbers'].str.replace("[]]","")

new_df['playerId'] = new_df['playerId'].str.replace("[[]","")
new_df['playerId'] = new_df['playerId'].str.replace("[]]","")

# CONVERT COLUMNS TO INTEGER
new_df['formationSlots'] = new_df['formationSlots'].astype(int)
new_df['jerseyNumbers'] = new_df['jerseyNumbers'].astype(int)
new_df['playerId'] = new_df['playerId'].astype(int)

# GET FIRST ELEMENT FROM A COLUMN LIST
new_df['startMinuteExpanded'] = new_df.startMinuteExpanded.apply(lambda x: x[0])
new_df['endMinuteExpanded'] = new_df.endMinuteExpanded.apply(lambda x: x[0])
new_df['formationId'] = new_df.formationId.apply(lambda x: x[0])

new_df[['formationId', 'formationSlots', 'startMinuteExpanded', 'endMinuteExpanded', 'jerseyNumbers', 'playerId']] = new_df[['formationId', 'formationSlots', 'startMinuteExpanded', 'endMinuteExpanded', 'jerseyNumbers', 'playerId']].astype(int)

# REMOVE VALUES EQUAL TO 0
new_df = new_df.loc[new_df.formationSlots != 0].reset_index(drop=True)

# CREATE COLUMN FORMATION BASED ON IF CONDITION
new_df['formationName'] = 'tactic'
for idx, row in enumerate(new_df.formationId):
    if row == 2:
        new_df['formationName'][idx] = '442'
    elif row == 3:
        new_df['formationName'][idx] = '41212 (Diamond)'
    elif row == 4:
        new_df['formationName'][idx] = '433'
    elif row == 5:
        new_df['formationName'][idx] = '451'
    elif row == 6:
        new_df['formationName'][idx] = '4411'
    elif row == 7:
        new_df['formationName'][idx] = '4141'
    elif row == 8:
        new_df['formationName'][idx] = '4231'
    elif row == 9:
        new_df['formationName'][idx] = '4321'
    elif row == 10:
        new_df['formationName'][idx] = '532'
    elif row == 11:
        new_df['formationName'][idx] = '541'
    elif row == 12:
        new_df['formationName'][idx] = '352'
    elif row == 13:
        new_df['formationName'][idx] = '343'
    elif row == 14:
        new_df['formationName'][idx] = '31312'
    elif row == 15:
        new_df['formationName'][idx] = '4222'
    elif row == 16:
        new_df['formationName'][idx] = '3511'
    elif row == 17:
        new_df['formationName'][idx] = '3421'
    elif row == 18:
        new_df['formationName'][idx] = '3412'        


# CREATE COLUMN POSITION BASED ON TWO IF CONDITION
new_df['position'] = 'POS'

formations = new_df.copy()

# formationId 2: 4-4-2
for index, formationId in enumerate(formations.formationId):
 if formations.formationSlots[index] == 0:
  formations.position[index] = 'Bench'
 elif formations.formationSlots[index] == 1:
  formations.position[index] = 'GK'
 elif (formationId == 2) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 2) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 2) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'RCM'
 elif (formationId == 2) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'RCB'
 elif (formationId == 2) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'LCB'
 elif (formationId == 2) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RM'
 elif (formationId == 2) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'LCM'
 elif (formationId == 2) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'LCF'
 elif (formationId == 2) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RCF'
 elif (formationId == 2) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LM'
 elif (formationId == 3) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 3) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 3) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'CDM'
 elif (formationId == 3) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'RCB'
 elif (formationId == 3) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'LCB'
 elif (formationId == 3) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RCM'
 elif (formationId == 3) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'CAM'
 elif (formationId == 3) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'LCF'
 elif (formationId == 3) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RCF'
 elif (formationId == 3) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LCM'
 elif (formationId == 4) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 4) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 4) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'CDM'
 elif (formationId == 4) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'RCB'
 elif (formationId == 4) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'LCB'
 elif (formationId == 4) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RCM'
 elif (formationId == 4) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'LCM'
 elif (formationId == 4) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CF'
 elif (formationId == 4) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RW'
 elif (formationId == 4) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LW'
 elif (formationId == 5) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 5) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 5) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'RDM'
 elif (formationId == 5) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'RCB'
 elif (formationId == 5) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'LCB'
 elif (formationId == 5) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RM'
 elif (formationId == 5) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'LDM'
 elif (formationId == 5) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CF'
 elif (formationId == 5) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'CAM'
 elif (formationId == 5) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LM'
 elif (formationId == 6) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 6) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 6) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'RCM'
 elif (formationId == 6) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'RCB'
 elif (formationId == 6) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'LCB'
 elif (formationId == 6) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RM'
 elif (formationId == 6) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'LCM'
 elif (formationId == 6) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CF'
 elif (formationId == 6) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'CAM'
 elif (formationId == 6) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LM'
 elif (formationId == 7) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 7) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 7) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'CDM'
 elif (formationId == 7) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'RCB'
 elif (formationId == 7) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'LCB'
 elif (formationId == 7) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RM'
 elif (formationId == 7) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'RCM'
 elif (formationId == 7) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CF'
 elif (formationId == 7) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'LCM'
 elif (formationId == 7) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LM'
 elif (formationId == 8) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 8) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 8) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'LDM'
 elif (formationId == 8) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'RCB'
 elif (formationId == 8) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'LCB'
 elif (formationId == 8) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RW'
 elif (formationId == 8) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'RDM'
 elif (formationId == 8) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CF'
 elif (formationId == 8) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'CAM'
 elif (formationId == 8) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LW'
 elif (formationId == 9) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 9) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 9) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'CDM'
 elif (formationId == 9) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'RCB'
 elif (formationId == 9) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'LCB'
 elif (formationId == 9) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'LCM'
 elif (formationId == 9) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'RCM'
 elif (formationId == 9) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CF'
 elif (formationId == 9) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RAM'
 elif (formationId == 9) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LAM'
 elif (formationId == 10) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 10) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 10) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'LCB'
 elif (formationId == 10) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'CCB'
 elif (formationId == 10) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'RCB'
 elif (formationId == 10) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RCM'
 elif (formationId == 10) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'CDM'
 elif (formationId == 10) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'LCF'
 elif (formationId == 10) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RCF'
 elif (formationId == 10) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LCM'
 elif (formationId == 11) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 11) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 11) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'LCB'
 elif (formationId == 11) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'CCB'
 elif (formationId == 11) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'RCB'
 elif (formationId == 11) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RM'
 elif (formationId == 11) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'RCM'
 elif (formationId == 11) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CF'
 elif (formationId == 11) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'LCM'
 elif (formationId == 11) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LM'
 elif (formationId == 12) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RWB'
 elif (formationId == 12) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LWB'
 elif (formationId == 12) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'LCB'
 elif (formationId == 12) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'CCB'
 elif (formationId == 12) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'RCB'
 elif (formationId == 12) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RDM'
 elif (formationId == 12) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'LDM'
 elif (formationId == 12) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'LCF'
 elif (formationId == 12) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RCF'
 elif (formationId == 12) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'CAM'
 elif (formationId == 13) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RWB'
 elif (formationId == 13) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LWB'
 elif (formationId == 13) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'LCB'
 elif (formationId == 13) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'CCB'
 elif (formationId == 13) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'RCB'
 elif (formationId == 13) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RCM'
 elif (formationId == 13) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'LCM'
 elif (formationId == 13) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CF'
 elif (formationId == 13) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RW'
 elif (formationId == 13) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LW'
 elif (formationId == 14) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RCM'
 elif (formationId == 14) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LCM'
 elif (formationId == 14) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'CDM'
 elif (formationId == 14) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'CCB'
 elif (formationId == 14) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'RCB'
 elif (formationId == 14) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'LCB'
 elif (formationId == 14) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'CCM'
 elif (formationId == 14) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'RCF'
 elif (formationId == 14) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'CAM'
 elif (formationId == 14) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LCF'
 elif (formationId == 15) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 15) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 15) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'RDM'
 elif (formationId == 15) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'RCB'
 elif (formationId == 15) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'LCB'
 elif (formationId == 15) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'LDM'
 elif (formationId == 15) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'RCM'
 elif (formationId == 15) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'LCF'
 elif (formationId == 15) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RCF'
 elif (formationId == 15) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LCM'
 elif (formationId == 16) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RWB'
 elif (formationId == 16) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LWB'
 elif (formationId == 16) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'LCB'
 elif (formationId == 16) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'CCB'
 elif (formationId == 16) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'RCB'
 elif (formationId == 16) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RDM'
 elif (formationId == 16) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'LDM'
 elif (formationId == 16) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CF'
 elif (formationId == 16) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'CAM'
 elif (formationId == 16) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'CCM'
 elif (formationId == 17) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RWB'
 elif (formationId == 17) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LWB'
 elif (formationId == 17) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'LCB'
 elif (formationId == 17) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'CCB'
 elif (formationId == 17) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'RCB'
 elif (formationId == 17) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RCM'
 elif (formationId == 17) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'LCM'
 elif (formationId == 17) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'LAM'
 elif (formationId == 17) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RAM'
 elif (formationId == 17) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'CF'
 elif (formationId == 18) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RWB'
 elif (formationId == 18) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LWB'
 elif (formationId == 18) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'LCB'
 elif (formationId == 18) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'CCB'
 elif (formationId == 18) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'RCB'
 elif (formationId == 18) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RCM'
 elif (formationId == 18) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'LCM'
 elif (formationId == 18) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CAM'
 elif (formationId == 18) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RCF'
 elif (formationId == 18) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LCF'
        
new_df = formations[['formationId', 'formationSlots', 'startMinuteExpanded', 'endMinuteExpanded', 'jerseyNumbers', 'playerId', 'formationName', 'position']]

In [ ]:
len(new_df.loc[new_df.playerId == 136741]['position'].unique())

In [25]:
df3 = df2.loc[(df2.team == 'Portugal') & (df2.Match_ID == 13)].reset_index(drop=True)

In [26]:
df3.to_excel('data.xlsx')

In [6]:
formations = new_df[['playerId','position', 'startMinuteExpanded', 'endMinuteExpanded']]
#formations = formations[formations.position != 'Bench']
df = df3.merge(formations, on='playerId', how='left')
df.columns

Index(['Unnamed: 0', 'id', 'eventId', 'minute', 'second', 'teamId', 'x', 'y',
       'expandedMinute', 'qualifiers', 'satisfiedEventsTypes', 'isTouch',
       'period.value', 'period.displayName', 'type.value', 'typedisplayName',
       'outcomeType.value', 'outcomeTypedisplayName', 'playerId', 'endX',
       'endY', 'cardType.value', 'cardType.displayName', 'goalMouthZ',
       'goalMouthY', 'isGoal', 'isShot', 'relatedEventId', 'relatedPlayerId',
       'blockedX', 'blockedY', 'Match_ID', 'name', 'team', 'home_Team',
       'away_Team', 'League', 'First_Name', 'Last_Name', 'Abreviation',
       'matchTimestamp', 'position_x', 'position_y', 'startMinuteExpanded',
       'endMinuteExpanded'],
      dtype='object')

In [33]:
formations = new_df[['playerId', 'position', 'startMinuteExpanded', 'endMinuteExpanded']]
#formations = formations[formations.position != 'Bench']
df = df3.merge(formations, on='playerId', how='left')
df['Position'] = np.where(
    (df['minute'] >= df['startMinuteExpanded']) &
    (df['minute'] <= df['endMinuteExpanded']),
    df['position'],
    np.nan
)
df = df.drop(['endMinuteExpanded', 'startMinuteExpanded', 'Unnamed: 0'], axis=1)
for index, playerId in enumerate(df.playerId):
    if (playerId == 0):
        df.position_filled[index] = ''

df.drop_duplicates(subset=['name', 'matchTimestamp', 'team', 'typedisplayName', 'x', 'y'], keep='first', inplace=True)
df.sort_values(by=['Match_ID', 'matchTimestamp'], inplace=True, ascending=[True, True])
df.reset_index(drop=True, inplace=True)
df = df[['matchTimestamp', 'playerId', 'name', 'Position', 'team', 'typedisplayName', 'outcomeTypedisplayName', 'x', 'y', 'endX', 'endY', 'home_Team', 'away_Team', 'Match_ID']]
df

,matchTimestamp,playerId,name,Position,team,typedisplayName,outcomeTypedisplayName,x,y,endX,endY,home_Team,away_Team,Match_ID
0,1970-01-01 00:00:07,353418.0,Diogo Dalot,RB,Portugal,Aerial,Successful,35.6,28.2,0.0,0.0,Portugal,Switzerland,13
1,1970-01-01 00:00:07,353418.0,Diogo Dalot,RB,Portugal,Pass,Unsuccessful,30.9,24.8,41.7,25.8,Portugal,Switzerland,13
2,1970-01-01 00:00:09,123761.0,Bruno Fernandes,RW,Portugal,Aerial,Unsuccessful,45.3,25.0,0.0,0.0,Portugal,Switzerland,13
3,1970-01-01 00:00:10,99911.0,William Carvalho,CDM,Portugal,Pass,Successful,34.3,28.9,29.8,7.1,Portugal,Switzerland,13
4,1970-01-01 00:00:12,353418.0,Diogo Dalot,RB,Portugal,BallRecovery,Successful,29.8,7.1,0.0,0.0,Portugal,Switzerland,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,1970-01-01 01:33:12,384887.0,Vitinha,NaN,Portugal,Pass,Unsuccessful,89.8,15.5,93.2,22.0,Portugal,Switzerland,13
708,1970-01-01 01:33:14,384887.0,Vitinha,NaN,Portugal,CornerAwarded,Successful,97.1,25.8,0.0,0.0,Portugal,Switzerland,13
709,1970-01-01 01:33:43,384887.0,Vitinha,NaN,Portugal,Pass,Successful,99.5,0.5,89.6,48.1,Portugal,Switzerland,13
710,1970-01-01 01:33:45,313171.0,Rúben Dias,NaN,Portugal,SavedShot,Successful,89.5,48.1,0.0,0.0,Portugal,Switzerland,13


In [34]:
df.loc[df.name == 'Bernardo Silva']['playerId'].unique()

array([136741.])